### Break up Excel File to Multiple CSV Flat Files

We don't have to execute this section of codes. Just trying out some stuff, how to break up excel workbook with multiple worksheets into distinct CSV files

In [ ]:
excelFileName = 'Financial_Data'
# PATH_TO_POWERSHELL_EXECUTABLE = r"C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe"
# PATH_TO_POWERSHELL_SCRIPT = r"C:\Users\stanley.setiawan\Documents\Lessons\PowerShell\ConvertExcelToCSV\convertToCSV.ps1"
# PATH_TO_POWERSHELL_EXECUTABLE = r"powershell"
# PATH_TO_POWERSHELL_SCRIPT = r"./convertToCSV.ps1"

In [ ]:

# subprocess.run(['powershell.exe','convertToCSV.ps1',excelFileName])    
# subprocess.run([PATH_TO_POWERSHELL_EXECUTABLE,PATH_TO_POWERSHELL_SCRIPT,excelFileName])    
subprocess.run([r'powershell','./convertToCSV.ps1',excelFileName])    

### Import all the Necessary Modules

In [1]:
import pandas as pd
import subprocess
import re
import pyodbc
from collections import namedtuple
import numpy as np

### Create Test Dataframe 

In [18]:
test_df = pd.DataFrame({'Column_1':['Stanley','Leonhard','Setiawan',None],
                        'Column_2':['22','This is it','random',None],
                        'Column_3':['11/22/2017','12/20/2018','09/05/2020',None],
                        'Column_4':['10/27/2015','12/15/2016','10/31/2015',None]})

test_df['Column_5'] = pd.Series(['Jan','Feb','Mar',None])
test_df['Column_6'] = pd.Series(['January','February','March',None])
test_df['Column_7'] = [22,None,'',None]
test_df['Column_8'] = [3.5,0.05,999.4,None]
test_df['Column_9'] = ['15:00','03:00','09:00',None]
test_df['Column_10'] = ['   ',' stanley this ',' ',"\n \n \t some'String"]

In [19]:
test_df

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10
0,Stanley,22,11/22/2017,10/27/2015,Jan,January,22,3.50,15:00,
1,Leonhard,This is it,12/20/2018,12/15/2016,Feb,February,None,0.05,03:00,stanley this
2,Setiawan,random,09/05/2020,10/31/2015,Mar,March,,999.40,09:00,
3,None,None,None,None,None,None,None,NaN,None,\n \n \t some'String


## Cleaning up Data

In [ ]:
# drop all rows which contain em
# Adj_Close_df.dropna(how='all',inplace=True)

### Function Definitions

In [3]:
# ============================================================================
# Function to find out data type of a dataframe column (Pandas Series Object) 
# ============================================================================

def findDataType(dataFrameColumn):
    try:
        column = pd.to_numeric(dataFrameColumn)
        dataType=column.dtype.name
    except:
        try:
            column = pd.to_datetime(dataFrameColumn)
        except:
            column = dataFrameColumn.astype('string')
            dataType = column.dtype.name
        else:
            if (column.dt.floor('d') == column).all():
                dataType = 'date'
            elif (column.dt.date == pd.Timestamp('now')).all():
                dataType = 'time'
            else:
                dataType=column.dtype.name
    finally:
        return dataType
        

In [4]:

# ==================================================================
# Function to convert the dataframe data types to SQL Data Types 
# ====================================================================

def getSQLTableStructure(dataframe):
    sqlTableStructure = {}
    sqlColumn = namedtuple('sqlColumn',[
         'sqlDataType',
         'noOfNulls',
         'allowNull'
    ])

    dataFrameStructure = getDataframeStructure(dataframe)
    for columnName,namedTupleObj in dataFrameStructure.items():

        #-----------------------------------------------------
        # Parse integer to its corresponding data type in SQL
        #-----------------------------------------------------
        if 'int' in namedTupleObj.dataType:  

            #get the number of bits used 
            no_of_bits = re.search('\d+',namedTupleObj.dataType).group()

            if no_of_bits == '32':
                sqlDataType = 'int'
            elif no_of_bits == '64':
                sqlDataType = 'bigint'
            elif no_of_bits == '16':
                sqlDataType = 'smallint'
            elif no_of_bits == '8':
                sqlDataType = 'tinyint'

        elif 'float' in namedTupleObj.dataType:

            sqlDataType = 'float'

        elif 'date' == namedTupleObj.dataType:

            sqlDataType = 'date'

        elif 'time' == namedTupleObj.dataType:

            sqlDataType = 'time'

        elif 'datetime' in namedTupleObj.dataType:

            sqlDataType = 'datetime'

        elif 'string' == namedTupleObj.dataType:

            # get the max number of bytes required for this column
            maxByteSize = dataframe[columnName].apply(lambda eachString: len(eachString.encode('UTF-8'))).max()
            sqlDataType = f'varchar({maxByteSize})'

            # need more functionality here to detect fixed string column

        # add a new sqlColumn to sqlTableStructure dictionary
        # based on the converted data type
        sqlTableStructure[columnName] = sqlColumn(sqlDataType=sqlDataType,
                                                  noOfNulls=namedTupleObj.noEmptyCells,
                                                  allowNull=namedTupleObj.allowNull)
    
    return (sqlTableStructure,dataFrameStructure)


In [5]:

# ===========================================================
# Get Data type for each column of pandas dataframe
# =========================================================

def getDataframeStructure(dataframe):
    sql_table_structure = {}
    sqlColumn = namedtuple('sqlColumn',[
        'dataType',
        'noEmptyCells',
        'allowNull' 
    ])

    for columnName in dataframe.columns:

        column = dataframe[columnName]

        # Get the row indexes which contain empty value/empty string or None/Null
        empty_cells_indexes = column[(column.isna()) | 
                                     (column == '') |
                                     (column.astype(str).str.isspace())].index
        
        # assign 'NULL' string to dataframe values that is either:
        # - a 'None'
        # - an empty string ''
        # - just a string that contains whitespaces '    '
        dataframe[columnName][empty_cells_indexes] = 'NULL'

        allowNull = '' if empty_cells_indexes.size > 0 else 'NOT NULL' 

        # drop the empty_cells_indexes from the column (if there's any)
        cleaned_column = column.drop(labels=empty_cells_indexes)

        # ----------------------------------------------------- 
        # make a best guess what the data type of the column is 
        # ----------------------------------------------------- 

        sql_table_structure[columnName] = sqlColumn(dataType=findDataType(cleaned_column),
                                                    noEmptyCells=empty_cells_indexes.size,
                                                    allowNull=allowNull)
        
    # return a dictionary containing the implied basic SQL Table Structure
    return sql_table_structure



In [6]:
# ===========================================================
# Generate the SQL String which will create a table
# =========================================================

def createTableQueryString(dbName,schema,tableName,sqlTableStructure):
    queryString = f'CREATE TABLE {dbName}.{schema}.{tableName}('
    
    for columnName,sqlColumn in sqlTableStructure.items():
        queryString += f'\n"{columnName}" {sqlColumn.sqlDataType} {sqlColumn.allowNull},'
    
    queryString += ');'
    return queryString



In [7]:
# ===========================================================
# Generate SQL query to insert values into database talbe
# =========================================================

def insertValuesQueryString(dbName,schema,tableName,dataframe):
    
    # get a list of columnNames from dataframe
    columnNamesList = ','.join([f'"{columnName}"' for columnName in dataframe.columns]) 
    queryString = f'INSERT INTO {dbName}.{schema}.{tableName} ({columnNamesList}) \n VALUES \n' 
    
    for index,*values in dataframe.itertuples():
        if index == dataframe.index[-1]:
            queryString += '(' + ','.join(["'" + str(value).replace("'","''") + "'" 
                                           if str(value) != 'NULL' else 'NULL'
                                           for value in values]) + ')'
        else:
            queryString += '(' + ','.join(["'" + str(value).replace("'","''") + "'" 
                                           if str(value) != 'NULL' else 'NULL' 
                                           for value in values]) + '),\n'
    return queryString
    

In [8]:
indexes = np.where((test_df.isna()) |  
                    (test_df == '') |
                    (test_df.apply(lambda row: row.str.isspace(), axis = 1)))

for rowNum,colNum in zip(indexes[0],indexes[1]):
    test_df.iloc[rowNum,colNum] = 'NULL'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


## Import other dataframes

This section is optional, it's just to import other dataframes if we want to 

In [ ]:
test_df = pd.read_csv('Boston_Crimes.csv',encoding='latin')

## Connect to Microsoft SQL Server

In [9]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=KA-PF14P8C2\SQLEXPRESS;'
                      'Database=MBAN;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [20]:
createTableString = createTableQueryString('MBAN','dbo','test',getSQLTableStructure(test_df)[0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [21]:
insertValueString = insertValuesQueryString('MBAN','dbo','test',test_df)

In [22]:
print(createTableString)

CREATE TABLE MBAN.dbo.test(
"Column_1" varchar(8) ,
"Column_2" varchar(10) ,
"Column_3" date ,
"Column_4" date ,
"Column_5" varchar(4) ,
"Column_6" varchar(8) ,
"Column_7" bigint ,
"Column_8" float ,
"Column_9" time ,
"Column_10" varchar(17) ,);


In [23]:
print(insertValueString)

INSERT INTO MBAN.dbo.test ("Column_1","Column_2","Column_3","Column_4","Column_5","Column_6","Column_7","Column_8","Column_9","Column_10") 
 VALUES 
('Stanley','22','11/22/2017','10/27/2015','Jan','January','22','3.5','15:00',NULL),
('Leonhard','This is it','12/20/2018','12/15/2016','Feb','February',NULL,'0.05','03:00',' stanley this '),
('Setiawan','random','09/05/2020','10/31/2015','Mar','March',NULL,'999.4','09:00',NULL),
(NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,'
 
 	 some''String')


In [24]:
cursor.execute(createTableString)

In [25]:
cursor.execute(insertValueString)

In [ ]:
# with open ('file.txt','w') as textFile:
#     textFile.write(insertValueString)

In [26]:
conn.commit()

In [45]:
pyodbc.version

'4.0.0-unsupported'